# T1221 - Template Injection
Adversaries may create or modify references in user document templates to conceal malicious code or force authentication attempts. For example, Microsoft’s Office Open XML (OOXML) specification defines an XML-based format for Office documents (.docx, xlsx, .pptx) to replace older binary formats (.doc, .xls, .ppt). OOXML files are packed together ZIP archives compromised of various XML files, referred to as parts, containing properties that collectively define how a document is rendered.(Citation: Microsoft Open XML July 2017)

Properties within parts may reference shared public resources accessed via online URLs. For example, template properties may reference a file, serving as a pre-formatted document blueprint, that is fetched when the document is loaded.

Adversaries may abuse these templates to initially conceal malicious code to be executed via user documents. Template references injected into a document may enable malicious payloads to be fetched and executed when the document is loaded.(Citation: SANS Brian Wiltse Template Injection) These documents can be delivered via other techniques such as [Phishing](https://attack.mitre.org/techniques/T1566) and/or [Taint Shared Content](https://attack.mitre.org/techniques/T1080) and may evade static detections since no typical indicators (VBA macro, script, etc.) are present until after the malicious payload is fetched.(Citation: Redxorblue Remote Template Injection) Examples have been seen in the wild where template injection was used to load malicious code containing an exploit.(Citation: MalwareBytes Template Injection OCT 2017)

Adversaries may also modify the <code>*\template</code> control word within an .rtf file to similarly conceal then download malicious code. This legitimate control word value is intended to be a file destination of a template file resource that is retrieved and loaded when an .rtf file is opened. However, adversaries may alter the bytes of an existing .rtf file to insert a template control word field to include a URL resource of a malicious payload.(Citation: Proofpoint RTF Injection)(Citation: Ciberseguridad Decoding malicious RTF files)

This technique may also enable [Forced Authentication](https://attack.mitre.org/techniques/T1187) by injecting a SMB/HTTPS (or other credential prompting) URL and triggering an authentication attempt.(Citation: Anomali Template Injection MAR 2018)(Citation: Talos Template Injection July 2017)(Citation: ryhanson phishery SEPT 2016)

## Atomic Tests

### Atomic Test #1 - WINWORD Remote Template Injection
Open a .docx file that loads a remote .dotm macro enabled template from https://github.com/redcanaryco/atomic-red-team/tree/master/atomics/T1221/src/opencalc.dotm 
Executes the code specified within the .dotm template.
Requires download of WINWORD found in Microsoft Ofiice at Microsoft: https://www.microsoft.com/en-us/download/office.aspx.  
Default docs file opens Calculator.exe when test sucessfully executed, while AV turned off.

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: 
##### Check Prereq Commands:
```cmd
None
```
##### Get Prereq Commands:
```cmd
None
```

In [ ]:
Invoke-AtomicTest T1221 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
start PathToAtomicsFolder\T1221\src\Calculator.docx
```

In [ ]:
Invoke-AtomicTest T1221 -TestNumbers 1

## Detection
Analyze process behavior to determine if user document applications (such as Office) are performing actions, such as opening network connections, reading files, spawning abnormal child processes (ex: [PowerShell](https://attack.mitre.org/techniques/T1059/001)), or other suspicious actions that could relate to post-compromise behavior.

Monitor .rtf files for strings indicating the <code>&#42;\template</code> control word has been modified to retrieve a URL resource, such as <code>&#42;\template http</code> or <code>&#42;\template \u-</code>.


## Shield Active Defense
### Decoy System 
 Configure a computing system to serve as an attack target or experimental environment.   

 A decoy system is a computing resource presented to the adversary in support of active defense.  The underlying system can be real, virtual, or simulated, and can be presented as one of a variety of IT devices including user workstations, servers, networking systems, IOT (embedded devices), mobile systems like phones, etc.
#### Opportunity
There is an opportunity to study the adversary and collect first-hand observations about them and their tools.
#### Use Case
A defender can have decoy systems that are easy to gain access to and have Office installed. The decoy system can be monitored to see if an adversary attempts to inject anything malicious into Office templates.
#### Procedures
Use an isolated system to visit a suspected compromised website.  Collect any associated scripting code or files dropped onto the system.
Setup a server which appears to be something that is commonly expected within a network, such as web server.
